In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [4]:
model_names = ["Vilt", "Blip_base", "Blip_large", "GiT_base", "GiT_large"]

image_names = ["Gaussian_5_2", "Gaussian_5_3", "Gaussian_5_4", "Gaussian_5_5",
               "white_house_brown_cat", "white_house_brown_cow", "white_house_brown_dog", "white_house_brown_horse",
               "white_house_green_cat", "white_house_green_unknown",
               "white_house_pink_cow", "white_house_pink_unknown",
               "white_house_purple_dog", "white_house_purple_horse",
               "white_house_red_duck", "white_house_red_unknown",
               "white_house_yellow_unknown"]

model_type = "VQA"

criteria = ["image_type", "used_language", "specificity", "question_length", "complexity", "image_relatedness"]
fixed_cols = ["question", "answer_gt"] + criteria

In [5]:
np_logits_file_path = f"./{model_type}_test_logits.npy"
logits_dict = np.load(np_logits_file_path, allow_pickle=True).item()
logits_dict.keys()

dict_keys(['Vilt', 'Blip_base', 'Blip_large', 'GiT_base', 'GiT_large'])

In [6]:
def T_1(logits):
  N = len(logits)
  x_max, x_min = np.max(logits), np.min(logits)
  return (x_max-x_min)/np.log(N)

def T_2(logits_list, sigma):
  t_min_list = list()
  for logits in logits_list:
    t_min = list()
    for logit in logits:
      x_max, x_min = np.max(logit), np.min(logit)
      t_temp = (x_max-x_min)/np.log(1+sigma)
      t_min.append(t_temp)
    t_min_list.append(np.min(t_min)) 
  return np.min(t_min_list)

def softmax_stable(x, T=1):
  return (np.exp((x - np.max(x))/T) / np.exp((x - np.max(x))/T).sum())

In [7]:
sigmas = [i/100 for i in range(1, 201)]
T_max_2_dict = np.load("./T_max_2_dict.npy", allow_pickle=True).item()
T_max_2_dict.keys()

dict_keys(['Vilt', 'Blip_base', 'Blip_large', 'GiT_base', 'GiT_large'])

In [8]:
sigmas_cols = list()
for sigma in sigmas:
    sigmas_cols.append(f"T_2_{sigma}")
    sigmas_cols.append(f"P_T_2_{sigma}")
len(sigmas_cols)

400

In [9]:
df_fixed_cols = ["image_name"]+fixed_cols
full_df_columns = df_fixed_cols + ["model_name", "full_answer", "clean_answer", "no_tokens", "correct", "Prob", "T_1", "P_T_1"]
full_df_columns.extend(sigmas_cols)
full_df_columns

['image_name',
 'question',
 'answer_gt',
 'image_type',
 'used_language',
 'specificity',
 'question_length',
 'complexity',
 'image_relatedness',
 'model_name',
 'full_answer',
 'clean_answer',
 'no_tokens',
 'correct',
 'Prob',
 'T_1',
 'P_T_1',
 'T_2_0.01',
 'P_T_2_0.01',
 'T_2_0.02',
 'P_T_2_0.02',
 'T_2_0.03',
 'P_T_2_0.03',
 'T_2_0.04',
 'P_T_2_0.04',
 'T_2_0.05',
 'P_T_2_0.05',
 'T_2_0.06',
 'P_T_2_0.06',
 'T_2_0.07',
 'P_T_2_0.07',
 'T_2_0.08',
 'P_T_2_0.08',
 'T_2_0.09',
 'P_T_2_0.09',
 'T_2_0.1',
 'P_T_2_0.1',
 'T_2_0.11',
 'P_T_2_0.11',
 'T_2_0.12',
 'P_T_2_0.12',
 'T_2_0.13',
 'P_T_2_0.13',
 'T_2_0.14',
 'P_T_2_0.14',
 'T_2_0.15',
 'P_T_2_0.15',
 'T_2_0.16',
 'P_T_2_0.16',
 'T_2_0.17',
 'P_T_2_0.17',
 'T_2_0.18',
 'P_T_2_0.18',
 'T_2_0.19',
 'P_T_2_0.19',
 'T_2_0.2',
 'P_T_2_0.2',
 'T_2_0.21',
 'P_T_2_0.21',
 'T_2_0.22',
 'P_T_2_0.22',
 'T_2_0.23',
 'P_T_2_0.23',
 'T_2_0.24',
 'P_T_2_0.24',
 'T_2_0.25',
 'P_T_2_0.25',
 'T_2_0.26',
 'P_T_2_0.26',
 'T_2_0.27',
 'P_T_2_0.27',

In [13]:
full_df = pd.read_excel(f"./{model_type}_full_results.xlsx", sheet_name=f"{model_type}_results")[full_df_columns]
full_df.head()

,image_name,question,answer_gt,image_type,used_language,specificity,question_length,complexity,image_relatedness,model_name,...,T_2_1.96,P_T_2_1.96,T_2_1.97,P_T_2_1.97,T_2_1.98,P_T_2_1.98,T_2_1.99,P_T_2_1.99,T_2_2.0,P_T_2_2.0
0,white_house_brown_cow,what is the brown object?,cow,0,0,0,0.0,0.0,0.0,Vilt,...,12.891233,0.0011746683+,12.851292,0.001179341+,12.811731,0.0011840159+,12.772544,0.0011886931+,12.733726,0.0011933725+
1,white_house_brown_cow,what is in the image with the cow?,house+sky,0,0,0,0.0,0.0,0.0,Vilt,...,12.891233,0.0008072921+,12.851292,0.0008095732+,12.811731,0.00081185263+,12.772544,0.00081413065+,12.733726,0.0008164071+
2,white_house_brown_cow,what is in the image with the house?,cow+sky,0,0,0,0.0,0.0,0.0,Vilt,...,12.891233,0.0010478273+,12.851292,0.0010516366+,12.811731,0.0010554465+,12.772544,0.001059257+,12.733726,0.0010630679+
3,white_house_brown_cow,what is the cow's color in the image?,brown,0,0,0,0.0,0.0,0.0,Vilt,...,12.891233,0.0013809956+,12.851292,0.0013871972+,12.811731,0.0013934046+,12.772544,0.0013996183+,12.733726,0.0014058386+
4,white_house_brown_cow,what is the house's color in the image?,white,0,0,0,0.0,0.0,0.0,Vilt,...,12.891233,0.0014833155+,12.851292,0.0014903083+,12.811731,0.0014973095+,12.772544,0.0015043196+,12.733726,0.001511338+


In [14]:
def str_to_float(str_):
    F = str_.split("+")
    F = [float(f) for f in F]
    return F

def seq_P(Ps):
    P = 1
    for p in Ps:
        P = P * p
    return P